In [1]:
"""
This notebook plots predicted and observed tracks for the rs185220 tiQTL (Fig. 3C)
"""

'\nThis notebook plots predicted and observed tracks for the rs185220 tiQTL (Fig. 3C)\n'

In [2]:
import numpy as np
import pyfastx
import os
import h5py
import sys
sys.path.append("../")
from utils import plot_side

In [3]:
# PRINT COMMANDS TO DOWNLOAD AND EXTRACT DATA

# Set SCRATCH to where you want to download data to
SCRATCH = "/Users/adamhe/github/scratch"

URL = "https://zenodo.org/records/10597358/files"
TAR = "example_tracks_and_deepshap.tar.gz"
print(f"wget {URL}/{TAR} -P {SCRATCH}")
print(f"tar -xvzf {SCRATCH}/{TAR} -C {SCRATCH}")

wget https://zenodo.org/records/10597358/files/example_tracks_and_deepshap.tar.gz -P /Users/adamhe/github/scratch
tar -xvzf /Users/adamhe/github/scratch/example_tracks_and_deepshap.tar.gz -C /Users/adamhe/github/scratch


In [4]:
# Load data

# Experimental
y = np.load(
    os.path.join(SCRATCH, "example_tracks_and_deepshap/concat_procap.npz")
)["arr_0"][:, np.r_[250:750, 1250:1750]]

# Predicted
prediction = h5py.File(os.path.join(SCRATCH, "example_tracks_and_deepshap/fold_1_examples_prediction.h5"))
tracks = prediction["track"]
quantity = prediction["quantity"]
quantity = 10 ** (np.hstack(
    [np.log10(quantity[:]), np.ones(quantity.shape[0]).reshape(-1, 1)]
) @ np.array([[1.59687745, -0.80203685]]).T)
y_norm = tracks / np.array(tracks).sum(axis=1, keepdims=True)
y_pred_scaled = y_norm * quantity

/var/folders/sd/3462tpx12fbfxc_f8swcwyf40000gp/T/ipykernel_99620/3207020389.py:15: RuntimeWarning: invalid value encountered in divide
  y_norm = tracks / np.array(tracks).sum(axis=1, keepdims=True)


In [6]:
# Divide individuals by genotype:

fasta = pyfastx.Fasta(os.path.join(SCRATCH, "example_tracks_and_deepshap/concat_sequence.fna.gz"))
seq_coords = [seq.name.split("_")[-1] for seq in fasta]

rs185220_coord = "chr5:56909030-56910029"
rs185220_seqs = [i for i in range(len(fasta)) if seq_coords[i] == rs185220_coord]

a_pred = [y_pred_scaled[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "A"]
a_expt = [y[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "A"]
ag_pred = [y_pred_scaled[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "R"]
ag_expt = [y[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "R"]
g_pred = [y_pred_scaled[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "G"]
g_expt = [y[i, :] for i in rs185220_seqs if fasta[i].seq[500] == "G"]

# Get mean per genotype

a_pred_mean = np.mean(np.array(a_pred), axis=0)
a_expt_mean = np.mean(np.array(a_expt), axis=0)
ag_pred_mean = np.mean(np.array(ag_pred), axis=0)
ag_expt_mean = np.mean(np.array(ag_expt), axis=0)
g_pred_mean = np.mean(np.array(g_pred), axis=0)
g_expt_mean = np.mean(np.array(g_expt), axis=0)

In [11]:
plot_side(
    a_pred_mean, ylim=[-1.5, 3], yticks=[0, 3],
    pic_name="img/model_fold_1_rs185220A_pred.pdf"
)

In [12]:
plot_side(
    a_expt_mean, ylim=[-1.5, 1], yticks=[0, 1],
    pic_name="img/model_fold_1_rs185220A_expt.pdf"
)

In [13]:
plot_side(
    g_pred_mean, ylim=[-1.5, 3], yticks=[0, 3],
    pic_name="img/model_fold_1_rs185220G_pred.pdf"
)

In [14]:
plot_side(
    g_expt_mean, ylim=[-1.5, 3], yticks=[0, 3],
    pic_name="img/model_fold_1_rs185220G_expt.pdf"
)

In [17]:
plot_side(
    ag_expt_mean, ylim=[-1.5, 3], yticks=[0, 3],
    pic_name="img/model_fold_1_rs185220AG_expt.pdf"
)

In [18]:
plot_side(
    ag_pred_mean, ylim=[-1.5, 3], yticks=[0, 3],
    pic_name="img/model_fold_1_rs185220AG_pred.pdf"
)

In [6]:
def l2_score(x, y, pseudocount=1e-5):
    return np.sqrt(np.sum(np.square(x - y), axis=-1))

l2_score(a_pred_mean, g_pred_mean)

3.1563125

In [8]:
l2_score(a_expt_mean, g_expt_mean)

4.29